<a href="https://colab.research.google.com/github/AnthonyTsiantis/Boeing-Innovation-Group1/blob/main/classifier_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sklearn
import sklearn.model_selection
import sklearn.svm
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tqdm
import os
import pandas as pd
import joblib

# Load Data (files need to be uploaded to colab first)

In [6]:
def get_df(datapath):
      frames = []
      df = pd.read_csv(f'{datapath}/Notre Dame Project Data - 1 of 10.csv', header=None)
      headers = df.iloc[0].tolist()
      df = df.drop(0)
      for file in [f'{datapath}/Notre Dame Project Data - {i} of 10.csv' for i in range(2, 11)]:
          df = pd.read_csv(file, header=None)
          df = df.drop(index=0)
          frames.append(df)
      dataset = pd.concat(frames)
      dataset.columns = headers
      return dataset

dataset = get_df('.')

# wuc seems to be like an ID number
print(dataset)

        rcvd_date_time buserno    wuc          emt m3_hrs m5_hrs wp_hrs  \
1      1/28/2023 14:06  538005  11101  0.983333333      0      0      0   
2      1/28/2023 14:06  538000  11101  1.383333333    0.5      2      0   
3      1/28/2023 14:08  538000  11201  1.166666667   0.25      2      0   
4      1/28/2023 14:12  538003  11201  1.416666667      0      0      0   
5      1/28/2023 14:13  538004  11201          1.2      0      2      0   
...                ...     ...    ...          ...    ...    ...    ...   
9996   9/30/2023 21:54  538009  11102          0.6      1      0      0   
9997   9/30/2023 21:56  538006  12320          0.8      0      0      0   
9998   9/30/2023 21:56  538004  11202         1.35      0      2      0   
9999   9/30/2023 21:59  538001  12320  2.166666667      0      0      0   
10000  9/30/2023 21:59  538006  11201         0.55      1      0      0   

      in_work_date_time       m4_hrs   comp_date_time  ... type_maint_code  \
1       1/28/2023 14:

# Text Embedding

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [ ]:
em_corr_action = model(dataset['corr_action'])
em_descrep_narrative = model(dataset['descrep_narrative'])
em_system_reason_desc = model(dataset['system_reason_desc'])

In [ ]:
# compile embedded vectors into data frame
em_data = pd.DataFrame()

em_data['corr_action'] = np.array(em_corr_action).tolist()
em_data['descrep_narrative'] = np.array(em_descrep_narrative).tolist()
em_data['system_reason_desc'] = np.array(em_system_reason_desc).tolist()

# SVC Classifier - one input column

In [ ]:
# testing and training on descrep_narrative data
# np.array() and .tolist() are used to tranform the encoder's output data from a tensor to a list
input_data = np.array(em_data['descrep_narrative']).tolist()


# classifier for column C
output_data = dataset["wuc"].copy()

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(input_data, output_data, test_size = 0.3, random_state=42)


In [ ]:
svc = sklearn.svm.SVC(random_state=42)

# train SVC model
svc.fit(X_train, y_train)

# test accuracy on SVC model
accuracy = svc.score(X_test, y_test)
print(f"{accuracy * 100}%")

96.35000000000001%


# SVC Classifier - all input columns

In [ ]:
# include all input data vectors in one data frame
em_df_exp = pd.DataFrame()

count = 0

import warnings
warnings.filterwarnings("ignore", message="DataFrame is highly fragmented.")

# expand data frame so each coefficient of input vector for each category has a column
for col in em_data.columns:
  data = {}
  for item in em_data[col]:
    count = 0
    for element in item:
      if count in data:
        data[count].append(element)
      else:
        data[count] = [element]
      count += 1

  for key in data:
    em_df_exp[f'{col}{key}'] = data[key].copy()

In [ ]:
print(em_df_exp)

       corr_action0  corr_action1  corr_action2  corr_action3  corr_action4  \
0         -0.052226      0.040350      0.059663      0.001304      0.062923   
1         -0.032559      0.032617      0.041037     -0.018566      0.060759   
2         -0.053364     -0.004635     -0.000591      0.013635     -0.028621   
3         -0.058994      0.026950      0.065375      0.030426      0.002120   
4         -0.008827     -0.050849     -0.006769      0.073079      0.008362   
...             ...           ...           ...           ...           ...   
19995     -0.058243      0.044362      0.062634      0.012688      0.049500   
19996     -0.032559      0.032617      0.041037     -0.018566      0.060759   
19997     -0.052226      0.040350      0.059663      0.001304      0.062923   
19998     -0.032559      0.032617      0.041037     -0.018566      0.060759   
19999     -0.058994      0.026950      0.065375      0.030426      0.002120   

       corr_action5  corr_action6  corr_action7  co

In [ ]:
# predict column C
output_data = dataset["wuc"].copy()

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(em_df_exp, output_data, test_size = 0.3, random_state=42)

svc = sklearn.svm.SVC(random_state=42)

# train SVC model
svc.fit(X_train, y_train)

# test accuracy on SVC model
accuracy = svc.score(X_test, y_test)
print(f"{accuracy * 100}%")
y_preds = svc.predict(X_test)
print(y_preds)
print(y_test)
""" Save Model
model_filename = 'svc_model.joblib'
joblib.dump(svc, model_filename)
"""
# no way 100% accuracy

100.0%
[15781 14106 19128 ... 13223 13195 11101]
650     15781
2041    14106
8668    19128
1114    14889
3902    11102
        ...  
4464    13284
5656    13373
9146    13223
567     13195
9165    11101
Name: wuc, Length: 6000, dtype: int64


['svc_model.joblib']

In [ ]:
# predict column X
output_data = dataset["wc_code"].copy()

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(em_df_exp, output_data, test_size = 0.3, random_state=42)

svc = sklearn.svm.SVC(random_state=35)

# train SVC model
svc.fit(X_train, y_train)

# test accuracy on SVC model
accuracy = svc.score(X_test, y_test)
print(f"{accuracy * 100}%")

# no way 100% accuracy

y_preds = svc.predict(X_test)
print(X_test)
print(y_preds)
print(y_test)
"""
single_row_to_predict = X_test.iloc[0:2, :]
print(single_row_to_predict)
single_row_pred = svc.predict(single_row_to_predict.values.reshape(2, -1))
print(single_row_pred)
print("y_test:",y_test)
"""

100.0%
       corr_action0  corr_action1  corr_action2  corr_action3  corr_action4  \
10650      0.038025      0.057936      0.063646     -0.036382      0.043500   
2041      -0.008594     -0.038299     -0.032309      0.075363      0.003945   

       corr_action5  corr_action6  corr_action7  corr_action8  corr_action9  \
10650     -0.057363      0.044513     -0.031037      0.053092      0.046949   
2041       0.063114      0.018948     -0.029736      0.059906      0.076808   

       ...  system_reason_desc502  system_reason_desc503  \
10650  ...               0.072607              -0.058618   
2041   ...               0.049453              -0.076500   

       system_reason_desc504  system_reason_desc505  system_reason_desc506  \
10650              -0.017408              -0.057954               0.079747   
2041                0.007846              -0.055741              -0.054582   

       system_reason_desc507  system_reason_desc508  system_reason_desc509  \
10650               0.0

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
